# Toxic Comment: Topic Modelling with pyLDAvis

The following material is inspired by jagangupta in the following link.
https://www.kaggle.com/jagangupta/understanding-the-topic-of-toxicity

In [1]:
import numpy as np
import pandas as pd
import string
import warnings

import re
import nltk
from nltk.corpus import stopwords
from string import punctuation
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
import gensim
from gensim.corpora import Dictionary
from nltk.stem.wordnet import WordNetLemmatizer 
import pyLDAvis.gensim
from gensim.models import LdaModel

#constants
eng_stopwords = set(stopwords.words("english"))
lem = WordNetLemmatizer()

#settings
warnings.filterwarnings("ignore")
start_time=time.time()
color = sns.color_palette()
sns.set_style("dark")

C:\Users\harri\Anaconda3\envs\py36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('C:\\Users\\harri\\.kaggle\\competitions\\jigsaw-toxic-comment-classification-challenge\\train.csv\\train.csv').fillna(' ')
test = pd.read_csv('C:\\Users\\harri\\.kaggle\\competitions\\jigsaw-toxic-comment-classification-challenge\\test.csv\\test.csv').fillna(' ')


In [3]:
df = pd.concat([train.iloc[:,0:2], test.iloc[:,0:2]])
df = df.reset_index(drop=True)

### Cleaning Function
Cleaning out leaky features and preprocessing our text data.

In [4]:
def clean(comment):
    #conv to lowercase
    comment = comment.lower()
    #replace new line
    comment = re.sub('\\n','',comment)
    #remove ip 
    comment = re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", "",comment)
    #remove username
    comment=re.sub("\[\[.*\]","",comment)
    #remove urls
    comment = re.sub("http://.*com", '', comment)
    #article ids
    comment = re.sub("\d:\d\d\s{0,5}$", '', comment)
    #tokenizer
    comment = gensim.utils.simple_preprocess(comment, deacc=True, min_len=3)
    return comment

df['comment_text'] = df['comment_text'].apply(clean)

In [5]:
all_text = df.comment_text
#Phrases help us group together bigrams :  new + york --> new_york
bigram = gensim.models.Phrases(all_text)

In [6]:
def seperate(word_list):
    """
    Function to clean the pre-processed word lists 
    
    Following transformations will be done
    1) Stop words removal from the nltk stopword list
    2) Bigram collation (Finding common bigrams and grouping them together using gensim.models.phrases)
    3) Lemmatization (Converting word to its root form : babies --> baby ; children --> child)
    """
    #remove stop words
    clean_words = [w for w in word_list if not w in eng_stopwords]
    #collect bigrams
    clean_words = bigram[clean_words]
    #Lemmatize
    clean_words=[lem.lemmatize(word, "v") for word in clean_words]
    return(clean_words)  

df['comment_text'] = df['comment_text'].apply(seperate)

In [7]:
all_text=df.comment_text
dictionary = Dictionary(all_text)
corpus = [dictionary.doc2bow(text) for text in all_text]

In [8]:
ldamodel = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.242444  0.032373       1        1  33.073174
7      0.167786 -0.019337       2        1  15.075710
5      0.115610 -0.132618       3        1   9.164503
6      0.081396  0.266042       4        1   8.651803
8      0.023324 -0.214742       5        1   8.355398
2      0.095038 -0.086706       6        1   7.238565
3      0.057621  0.217999       7        1   6.923871
1     -0.157000 -0.198571       8        1   5.216513
9     -0.345152  0.051698       9        1   3.213008
4     -0.281067  0.083863      10        1   3.087452, topic_info=       Category           Freq                       Term          Total  \
term                                                                       
46      Default  123537.000000                    article  123537.000000   
2       Default   48210.000000                       edit   48210.000000   
80      Default   64602.000000                  wikipedia   64602.000000   
787     Default   29956.000000                       fuck   29956.000000   
84      Default   61596.000000                       page   61596.000000   
11      Default   38272.000000                     please   38272.000000   
29      Default   38395.000000                      thank   38395.000000   
596     Default   18088.000000                      block   18088.000000   
170     Default   17072.000000                      image   17072.000000   
1204    Default   24464.000000                       name   24464.000000   
407     Default   34682.000000                        get   34682.000000   
2187    Default   14703.000000                       user   14703.000000   
16      Default   18643.000000                  talk_page   18643.000000   
59      Default   17760.000000                       list   17760.000000   
936     Default   31842.000000                     people   31842.000000   
279     Default   44126.000000                       like   44126.000000   
221412  Default   11930.000000                   wtf_omfg   11930.000000   
89      Default   50656.000000                        use   50656.000000   
131     Default   28345.000000                        add   28345.000000   
14      Default   11934.000000                     revert   11934.000000   
245790  Default    7100.000000                  phck_hole    7100.000000   
203     Default   32879.000000                     source   32879.000000   
8       Default   46764.000000                       make   46764.000000   
74      Default   47157.000000                      think   47157.000000   
208     Default    9682.000000                        tag    9682.000000   
304     Default   46158.000000                        say   46158.000000   
388     Default   32974.000000                       know   32974.000000   
69      Default   20476.000000                    section   20476.000000   
181     Default   20320.000000                       link   20320.000000   
197     Default   13566.000000                   question   13566.000000   
...         ...            ...                        ...            ...   
309703  Topic10     532.320251            hufgi_allumungi     533.187683   
3135    Topic10     520.729492            adding_nonsense     521.596558   
309384  Topic10     508.500214              peenus_peenus     509.367615   
319463  Topic10    1258.528687                cucks_cucks    1260.760254   
18826   Topic10     487.787048                      latin     488.654266   
4158    Topic10     545.419128                        www     546.408997   
9442    Topic10     457.537598                  committee     458.404877   
322885  Topic10     428.306641       naconkantari_fucking     429.174042   
322884  Topic10     428.306641             homo_lolololol     429.174042   
329580  Topic10    4120.892578                 jizz_farts    4133.884766   
312331  Topic10    1842.400757           